<a href="https://colab.research.google.com/github/drysmakhanova/Robt403/blob/main/IKLab7_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import numpy as np 
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers 
from keras.utils import np_utils 
from keras import backend as K 
import pandas as pd
from sklearn.model_selection import train_test_split
import random


In [136]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [138]:
data = pd.read_csv("dict1.csv", header = None, names = ["Angles", "XY"])


In [139]:
train = data['XY'].to_numpy()
labels = data['Angles'].to_numpy()

Here we use for labels translation and rotation values and keep only X,Y coordinaties and Rotation values.

In [140]:
X = list()
Y = list()
for i in range(len(train)):
    train[i] = train[i].strip('(array([').strip(']))')
    train[i] = train[i].replace(' ', '')
    train[i] = train[i].replace('  ', '')
    train[i] = train[i].replace(',0.25]),array([0.,0.,', ',')
    train[i] = train[i].replace(',2.5000e-01]),array([0.,0.,', ',')
    train[i] = train[i].replace(',0.25]),array([0.e+00,0.e+00,', ',')
    result = [float(val) for val in train[i].split(',')]
    result.pop()
    result.pop()
    X.append(result)
    labels[i] = labels[i].strip('(').strip(')')
    result = [float(val) for val in labels[i].split(',')]
    Y.append(result)

So Y values are 5  joint angles values of the robot and X are inputs: X,Y

Shuffling lists and keeping X-Y order

In [112]:
type(X[0])

list

In [141]:
temp = list(zip(X, Y))
random.shuffle(temp)
X,Y = zip(*temp)

In [142]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), np.asarray(Y), test_size=0.25)

In [143]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(375, 2)
(375, 5)
(125, 2)
(125, 5)


In [159]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
act = 'elu'
model = Sequential()
model.add(Dense(16, input_dim = 2, activation = act))
model.add(Dense(32, activation = act))
model.add(Dense(64, activation = act))
model.add(Dense(64, activation = act))
model.add(Dense(32, activation = act))
model.add(Dense(16, activation = act))
model.add(Dense(5, activation= act))
model.compile(loss=rmse, optimizer=Adam(0.001))

In [154]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_169 (Dense)           (None, 16)                48        
                                                                 
 dense_170 (Dense)           (None, 32)                544       
                                                                 
 dense_171 (Dense)           (None, 64)                2112      
                                                                 
 dense_172 (Dense)           (None, 64)                4160      
                                                                 
 dense_173 (Dense)           (None, 32)                2080      
                                                                 
 dense_174 (Dense)           (None, 16)                528       
                                                                 
 dense_175 (Dense)           (None, 5)               

In [155]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
12/12 [==============================] - 0s 2ms/step - loss: 0.5094
Epoch 2/20
12/12 [==============================] - 0s 2ms/step - loss: 0.3117
Epoch 3/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2850
Epoch 4/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2717
Epoch 5/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2749
Epoch 6/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2676
Epoch 7/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2598
Epoch 8/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2646
Epoch 9/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2717
Epoch 10/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2658
Epoch 11/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2612
Epoch 12/20
12/12 [==============================] - 0s 2ms/step - loss: 0.2711
Epoch 13/20
12/12 [==============================

In [156]:
scores = model.evaluate(X_test, y_test, verbose=0) 
print("RMSE: %.2f" % (scores))

RMSE: 0.26


In [157]:
print(model.predict(X_test[15].reshape(1,2)))
print(y_test[15])

[[-0.00531227 -0.22610939 -0.14534579  0.00699105 -0.16855258]]
[-0.39 -0.32 -0.38 -0.48  0.24]


Elu, Selu, Tanh show the best approximation